In [1]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, Dense, Concatenate, TimeDistributed, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
# First navigate to our directory
directory_path = os.path.join("Data", "synthetic_df.parquet")
# Load the dataframe
synthetic_df = pd.read_parquet(directory_path)

In [ ]:


# Define the input length (N)
N = 8192  # Example input length; adjust as needed

# Input layer
input_layer = Input(shape=(N, 1), name="Input")

# Inception-like layer with 1D convolutions
conv2 = Conv1D(16, kernel_size=2, activation='relu', padding='same', name="Conv_2")(input_layer)
conv4 = Conv1D(32, kernel_size=4, activation='relu', padding='same', name="Conv_4")(input_layer)
conv8 = Conv1D(64, kernel_size=8, activation='relu', padding='same', name="Conv_8")(input_layer)
conv32 = Conv1D(16, kernel_size=32, activation='relu', padding='same', name="Conv_32")(input_layer)
conv64 = Conv1D(8, kernel_size=64, activation='relu', padding='same', name="Conv_64")(input_layer)
conv128 = Conv1D(4, kernel_size=128, activation='relu', padding='same', name="Conv_128")(input_layer)
conv256 = Conv1D(4, kernel_size=256, activation='relu', padding='same', name="Conv_256")(input_layer)

# Concatenate the outputs of all convolutional layers
concat_layer = Concatenate(name="Inception_Concat")([conv2, conv4, conv8, conv32, conv64, conv128, conv256])

# Time Distributed Dense Layers
td_dense64 = TimeDistributed(Dense(64, activation='relu'), name="Dense_64")(concat_layer)
td_dense32 = TimeDistributed(Dense(32, activation='relu'), name="Dense_32")(td_dense64)
td_dense16 = TimeDistributed(Dense(16, activation='relu'), name="Dense_16")(td_dense32)

# Final layer with 4 outputs per input bin
output = TimeDistributed(Dense(4, activation=None), name="Output")(td_dense16)

# Define the model
model = Model(inputs=input_layer, outputs=output, name="Peak Picker V1")

# Compile the model
# Custom loss: binary cross-entropy for the first 2 nodes and MSE for the last 2 nodes
def custom_loss(y_true, y_pred):
    binary_loss = tf.keras.losses.binary_crossentropy(y_true[..., :2], y_pred[..., :2])
    regression_loss = tf.keras.losses.mean_squared_error(y_true[..., 2:], y_pred[..., 2:])
    return binary_loss + regression_loss

model.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss, metrics=["accuracy"])

# Summary
model.summary()

# Example data for training
# Inputs: shape (batch_size, N, 1)
# Outputs: shape (batch_size, N, 4)

# model.fit(X_train, y_train, batch_size=32, epochs=10)


Model: "Inception_TimeDistributed_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)  │ (None, 8192, 1)   │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv_2 (Conv1D)     │ (None, 8192, 16)  │         48 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv_4 (Conv1D)     │ (None, 8192, 32)  │        160 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv_8 (Conv1D)     │ (None, 8192, 64)  │        576 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv_32 (Conv1D)    │ (None, 8192, 16)  │        528 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv_64 (Conv1D)    │ (None, 8192, 8)   │        520 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv_128 (Conv1D)   │ (None, 8192, 4)   │        516 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Inception_Concat    │ (None, 8192, 140) │          0 │ Conv_2[0][0],     │
│ (Concatenate)       │                   │            │ Conv_4[0][0],     │
│                     │                   │            │ Conv_8[0][0],     │
│                     │                   │            │ Conv_32[0][0],    │
│                     │                   │            │ Conv_64[0][0],    │
│                     │                   │            │ Conv_128[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_64            │ (None, 8192, 64)  │      9,024 │ Inception_Concat… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_32            │ (None, 8192, 32)  │      2,080 │ Dense_64[0][0]    │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_16            │ (None, 8192, 16)  │        528 │ Dense_32[0][0]    │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Output              │ (None, 8192, 4)   │         68 │ Dense_16[0][0]    │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,048 (54.88 KB)

 Trainable params: 14,048 (54.88 KB)

 Non-trainable params: 0 (0.00 B)